In [3]:
import requests 
import pandas as pd
import pyodbc
from urllib import parse
from sqlalchemy import create_engine, text
import datetime
import os
import dotenv

# - Conexion a la Base de Datos AlpinaVista360
server= 'tcp:alpinasql03.database.windows.net'
database= 'AlpinaVista360'
username= 'alpinavistapru01'
password= 'Al$p1nC0lprudbu!'

# - Establecemos la conexión a la Base de Datos
params = ('DRIVER={ODBC Driver 17 for SQL Server}; \
                           SERVER='+ server +'; \
                           DATABASE='+ database +';\
                           UID='+ username +'; \
                           PWD='+password)

# - Cargamos variables de entorno con secretos
try:
    dotenv_path = dotenv.find_dotenv('storage/keys/.env')
    dotenv.load_dotenv(dotenv_path)

except:
    print('No hay variables locales!')

cnxn = pyodbc.connect(params)
cnxn.autocommit = True
query =  '''
            SELECT MAX(CAST(fecha_corte AS DATETIME2)) AS maxFechaCorte
            FROM x_dataFIC
         '''
maxFechaCorte = pd.read_sql(query, cnxn)

In [4]:
maxFechaCorte


,maxFechaCorte
0,2021-03-28


In [13]:
if maxFechaCorte.loc[0,'maxFechaCorte'] == None:
    
    # - fecha minima
    query = 'https://www.datos.gov.co/resource/qhpu-8ixx.json?$select=min(fecha_corte)'
    response = requests.get(query)
    
    if response:
        data = response.json()
        minDate = data[0].get('min_fecha_corte')
        print('Request exitoso! - la fecha minima del dataset es: ', minDate)
    else:
        print('Ha ocurrido un error en el Requests de la fecha minima')
    
    # - fecha maxima
    query = 'https://www.datos.gov.co/resource/qhpu-8ixx.json?$select=max(fecha_corte)'
    response = requests.get(query)
    
    if response:
        data = response.json()
        maxDate = data[0].get('max_fecha_corte')
        print('Request exitoso! - la fecha maxima del dataset es: ', maxDate)
    else:
        print('Ha ocurrido un error en el Requests de la fecha maxima')    
        
    # - Convertimos la fecha minima y maxima a formato datetime
    dt_minDate = datetime.datetime.strptime(minDate, '%Y-%m-%dT%H:%M:%S.%f')
    dt_maxDate = datetime.datetime.strptime(maxDate, '%Y-%m-%dT%H:%M:%S.%f')
    
    dt_minDate_inter = dt_minDate
    
    ciclo = 0
    while True:
        minDate_inter = dt_minDate_inter.strftime('%Y-%m-%dT%H:%M:%S.%f')
        dt_maxDate_inter = dt_minDate_inter + datetime.timedelta(days=90)
        maxDate_inter = dt_maxDate_inter.strftime('%Y-%m-%dT%H:%M:%S.%f')
        
        # - Realizamos la primera carga de datos a un df con fecha minima y maxima como parametros
        url = 'https://www.datos.gov.co/resource/qhpu-8ixx.json'
        filtros = f'?$where=fecha_corte between \'{minDate_inter}\' and \'{maxDate_inter}\'&$limit=1500000'
        query = url+filtros
        headers = {'X-App-Token':os.environ.get('X_APP_TOKEN')}
        response = requests.get(query, headers=headers)

        if response:
            data = response.json()
            df = pd.DataFrame(data)
            print('='*35)
            print('Request exitoso! - DataFrame cargado - dimensiones: ', df.shape)

        else:
            print('Ha ocurrido un error en el Requests de la carga de datos')
        
        # - Enviamos el lote a bd
        db_params = parse.quote_plus(params)
        engine = create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params), fast_executemany=True)
        df.to_sql(name='x_dataFIC',con=engine, index=False,if_exists='append',schema='dbo', chunksize = 10000)
        

        
        if dt_maxDate_inter > dt_maxDate:
            break
        else:
            dt_minDate_inter = dt_maxDate_inter + datetime.timedelta(days=1)
        
        print('contador de ciclo while: ',ciclo)
        print('fecha minima: ',minDate_inter)
        print('fecha maxima: ',maxDate_inter)      

Request exitoso! - la fecha minima del dataset es:  2016-01-01T00:00:00.000
Request exitoso! - la fecha maxima del dataset es:  2021-03-28T00:00:00.000
Request exitoso! - DataFrame cargado - dimensiones:  (42510, 26)


OperationalError: (pyodbc.OperationalError) ('08S01', '[08S01] [Microsoft][ODBC Driver 17 for SQL Server]Communication link failure (0) (SQLEndTran)')
(Background on this error at: http://sqlalche.me/e/14/e3q8)